In [ ]:
!pip install instructor

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 23.5 MB/s eta 0:00:00
  Attempting uninstall: jiter
    Found existing installation: jiter 0.7.1
    Uninstalling jiter-0.7.1:
      Successfully uninstalled jiter-0.7.1


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import json
import random
import instructor
import re
from pathlib import Path
from typing import List, TextIO

import openai
from datasets import load_dataset
# from fire import Fire
from pydantic import BaseModel
from tqdm import tqdm


def remove_substrings_with_double_angle_brackets(input_string):
    # Define the pattern to match substrings within double angled brackets
    pattern = r"<<[^>]+>>"
    # Use the sub() function from the re module to replace matching substrings with an empty string
    result = re.sub(pattern, "", input_string)
    return result


class ReasonSample(BaseModel):
    question: str
    explanation: str = ""
    answer: str = ""
    wrong_explanation: str = ""
    wrong_answer: str = ""
    pred: str = ""


class ReasonData(BaseModel):
    samples: List[ReasonSample]

    @classmethod
    def load(cls, path: str):
        samples = []
        with open(path) as f:
            for line in f:
                raw = json.loads(line)
                samples.append(ReasonSample(**raw))

        return cls(samples=samples)

    @classmethod
    def load_gsm8k_test(cls, path: str = "gsm8k", subset: str = "main", split="test"):
        samples = []
        for raw in load_dataset(path, subset, split=split):
            explanation, answer = raw["answer"].split("####")
            explanation = remove_substrings_with_double_angle_brackets(explanation)
            samples.append(
                ReasonSample(
                    question=raw["question"].strip(),
                    explanation=explanation.strip(),
                    answer=answer.strip(),
                )
            )

        return cls(samples=samples)

    @classmethod
    def load_gsm8k_incoherent_objects(cls, split: str = "test", sample: bool = False):
        if split == "train" and sample:
            samples = [
                ReasonSample(
                    question="There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?",
                    explanation="There are 15 trees originally. Then there were 21 trees after the Grove workers planted some more. So there must have been 21 - 15 = 6 trees that were planted.",
                    answer="6",
                    wrong_explanation="There are 21 - 15 = 6 trees originally. Then there were 15 trees after the Grove workers planted some more. So there must have been 21 trees that were planted.",
                    wrong_answer="21",
                ),
                ReasonSample(
                    question="If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot??",
                    explanation="There are originally 3 cars. Then 2 more cars arrive. Now 3 + 2 = 5 cars are in the parking lot.",
                    answer="5",
                    wrong_explanation="There are originally 3 + 2 = 5 cars. Then 3 more cars arrive. Now 2 cars are in the parking lot",
                    wrong_answer="2",
                ),
                ReasonSample(
                    question="Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?",
                    explanation="Originally, Leah had 32 chocolates and her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39 pieces left in total.",
                    answer="39",
                    wrong_explanation="Originally, Leah had 32 + 42 = 74 chocolates and her sister had 32. So in total they had 74 - 35 = 39. After eating 35, they had 42 pieces left in total",
                    wrong_answer="42",
                ),
                ReasonSample(
                    question="Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?",
                    explanation="Jason had 20 lollipops originally. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8 lollipops.",
                    answer="8",
                    wrong_explanation="Jason had 20 - 12 = 8 lollipops originally. Then he had 20 after giving some to Denny. So he gave Denny 12 lollipops",
                    wrong_answer="12",
                ),
            ]
            return cls(samples=samples)
        else:
            return cls.load_gsm8k_test()

    @classmethod
    def load_from_name(cls, name: str, **kwargs):
        if name == "gsm8k":
            return cls.load_gsm8k_incoherent_objects(**kwargs)
        else:
            raise KeyError(name)


class Prompter(BaseModel):
    def run(self, data_train: ReasonData, sample_test: ReasonSample) -> str:
        prompt = ""
        for sample in data_train.samples:
            prompt += f"Question: {sample.question}\n"
            prompt += f"Answer: {sample.answer}\n\n"

        prompt += f"Question: {sample_test.question}\n"
        prompt += "Answer: "
        return prompt

    @staticmethod
    def get_answer(text: str) -> str:
        parts = text.split("Answer: ")
        if len(parts) >= 2:
            return parts[1]
        else:
            return text


class ChainThoughtPrompter(Prompter):
    def run(self, data_train: ReasonData, sample_test: ReasonSample) -> str:
        prompt = ""
        for sample in data_train.samples:
            prompt += f"Question: {sample.question}\n"
            prompt += f"Explanation: {sample.explanation}\n"
            prompt += f"Answer: {sample.answer}\n\n"

        prompt += f"Question: {sample_test.question}\n"
        prompt += "Explanation: "
        return prompt

    def get_explanation(self, text: str) -> str:
        assert self is not None
        return text.split("\nAnswer: ")[0]


class ContrastiveChainThoughtPrompter(Prompter):
    def run(self, data_train: ReasonData, sample_test: ReasonSample) -> str:
        prompt = ""
        for sample in data_train.samples:
            prompt += f"Question: {sample.question}\n"
            prompt += f"Explanation: {sample.explanation}\n"
            prompt += f"Answer: {sample.answer}\n"
            prompt += f"Wrong explanation: {sample.wrong_explanation}\n"
            prompt += f"Wrong Answer: {sample.wrong_answer}\n\n"

        prompt += f"Question: {sample_test.question}\n"
        prompt += "Explanation: "
        return prompt


def select_prompter(name: str):
    if name == "standard":
        return Prompter()
    if name == "cot":
        return ChainThoughtPrompter()
    if name == "contrast_cot":
        return ContrastiveChainThoughtPrompter()
    else:
        raise KeyError(name)


class OpenAIModel(BaseModel):
    engine : str = ''
    use_azure: bool = False
    timeout: int = 60
    temperature: float = 0.0


    def run(self, prompt: str) -> str:
        output = ""
        error_message = "The response was filtered"
        while not output:
            try:
                key = "engine" if self.use_azure else "model"
                client = openai.OpenAI(api_key='sk-RsaFA9zGH4bOQl8ExoDoVfVTckmdGuOj2mgCGgRl0WfWQU8u',
                                      base_url="https://api.chatanywhere.tech/v1")
                response = client.chat.completions.create(model='gpt-4o-mini',

                    messages=[{"role": "user", "content": prompt}],
                    temperature=self.temperature,  # this is the degree of randomness of the model's output
                )
                if response.choices[0].finish_reason == "content_filter":
                    raise ValueError(error_message)
                output = response.choices[0].message.content
            except Exception as e:
                print(e)
                if error_message in str(e):
                    output = error_message

            if not output:
                print("OpenAIModel request failed, retrying.")

        return output


class Scorer(BaseModel):
    @staticmethod
    def run(pred: str, gold: str) -> float:
        pred = pred.replace("$", "")
        gold = gold.replace("$", "")
        return float(pred.startswith(gold))


def evaluate(
    model: OpenAIModel,
    data_train: ReasonData,
    data_test: ReasonData,
    prompter: Prompter,
    file: TextIO,
) -> dict:
    is_correct = []
    score = 0

    progress = tqdm(data_test.samples)
    sample: ReasonSample

    for sample in progress:
        prompt = prompter.run(data_train, sample)
        raw = model.run(prompt).strip()
        sample.pred = prompter.get_answer(raw)
        is_correct.append(Scorer.run(sample.pred, sample.answer))
        score = sum(is_correct) / len(is_correct)
        progress.set_postfix(score=score)
        print(dict(prompt=prompt, raw=raw, gold=sample.answer, pred=sample.pred))
        print(sample.json(), file=file)

    return dict(score=score)

In [ ]:

def main(
    data_name: 'gsm8k',
    prompt_name: 'contrast_cot',
    data_path: str = None,
    num_shots: int = 4,
    max_test_samples: int = 20,
    seed: int = 0,
    **kwargs,
):
    model = OpenAIModel()
    print(locals())

    data_train = ReasonData.load_from_name(data_name, split="train", sample=True)
    name = f"{prompt_name}_{num_shots=}_{max_test_samples=}"
    data_test = ReasonData.load_from_name(data_name, split="test")

    path_out = f"outputs/{data_name}/{kwargs.get('model_name')}/{name}.json"
    if num_shots >= 0:
        data_train.samples = data_train.samples[:num_shots]
    if 0 < max_test_samples < len(data_test.samples):
        random.seed(seed)
        data_test.samples = random.sample(data_test.samples, k=max_test_samples)
    print(dict(train=len(data_train.samples), test=len(data_test.samples)))

    Path(path_out).parent.mkdir(exist_ok=True, parents=True)
    with open(path_out, "w") as f:
        prompter = select_prompter(prompt_name)
        result = evaluate(model, data_train, data_test, prompter, file=f)

    print(path_out)
    print(result)
    return result["score"]


def infer(prompt_name: str, question: str, **kwargs):
    model = OpenAIModel(**kwargs)
    data = ReasonData.load_from_name("gsm8k", split="train", sample=True)
    sample = ReasonSample(question=question)
    prompt = select_prompter(prompt_name).run(data, sample)
    output = model.run(prompt)

    info = dict(question=question, prompt=prompt, output=output)
    print(json.dumps(info, indent=2))
    # return info



In [ ]:
main('gsm8k','contrast_cot')

{'data_name': 'gsm8k', 'prompt_name': 'contrast_cot', 'data_path': None, 'num_shots': 4, 'max_test_samples': 20, 'seed': 0, 'kwargs': {}, 'model': OpenAIModel(engine='', use_azure=False, timeout=60, temperature=0.0)}
{'train': 4, 'test': 20}


  5%|▌         | 1/20 [00:01<00:25,  1.34s/it, score=1]

{'prompt': 'Question: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\nExplanation: There are 15 trees originally. Then there were 21 trees after the Grove workers planted some more. So there must have been 21 - 15 = 6 trees that were planted.\nAnswer: 6\nWrong explanation: There are 21 - 15 = 6 trees originally. Then there were 15 trees after the Grove workers planted some more. So there must have been 21 trees that were planted.\nWrong Answer: 21\n\nQuestion: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot??\nExplanation: There are originally 3 cars. Then 2 more cars arrive. Now 3 + 2 = 5 cars are in the parking lot.\nAnswer: 5\nWrong explanation: There are originally 3 + 2 = 5 cars. Then 3 more cars arrive. Now 2 cars are in the parking lot\nWrong Answer: 2\n\nQuestion: Leah had 32 chocolates and her si

 10%|█         | 2/20 [00:03<00:36,  2.03s/it, score=1]

{'prompt': 'Question: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\nExplanation: There are 15 trees originally. Then there were 21 trees after the Grove workers planted some more. So there must have been 21 - 15 = 6 trees that were planted.\nAnswer: 6\nWrong explanation: There are 21 - 15 = 6 trees originally. Then there were 15 trees after the Grove workers planted some more. So there must have been 21 trees that were planted.\nWrong Answer: 21\n\nQuestion: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot??\nExplanation: There are originally 3 cars. Then 2 more cars arrive. Now 3 + 2 = 5 cars are in the parking lot.\nAnswer: 5\nWrong explanation: There are originally 3 + 2 = 5 cars. Then 3 more cars arrive. Now 2 cars are in the parking lot\nWrong Answer: 2\n\nQuestion: Leah had 32 chocolates and her si

...

 95%|█████████▌| 19/20 [00:47<00:02,  2.05s/it, score=0.789]

{'prompt': 'Question: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\nExplanation: There are 15 trees originally. Then there were 21 trees after the Grove workers planted some more. So there must have been 21 - 15 = 6 trees that were planted.\nAnswer: 6\nWrong explanation: There are 21 - 15 = 6 trees originally. Then there were 15 trees after the Grove workers planted some more. So there must have been 21 trees that were planted.\nWrong Answer: 21\n\nQuestion: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot??\nExplanation: There are originally 3 cars. Then 2 more cars arrive. Now 3 + 2 = 5 cars are in the parking lot.\nAnswer: 5\nWrong explanation: There are originally 3 + 2 = 5 cars. Then 3 more cars arrive. Now 2 cars are in the parking lot\nWrong Answer: 2\n\nQuestion: Leah had 32 chocolates and her si

100%|██████████| 20/20 [00:49<00:00,  2.48s/it, score=0.8]

{'prompt': 'Question: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\nExplanation: There are 15 trees originally. Then there were 21 trees after the Grove workers planted some more. So there must have been 21 - 15 = 6 trees that were planted.\nAnswer: 6\nWrong explanation: There are 21 - 15 = 6 trees originally. Then there were 15 trees after the Grove workers planted some more. So there must have been 21 trees that were planted.\nWrong Answer: 21\n\nQuestion: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot??\nExplanation: There are originally 3 cars. Then 2 more cars arrive. Now 3 + 2 = 5 cars are in the parking lot.\nAnswer: 5\nWrong explanation: There are originally 3 + 2 = 5 cars. Then 3 more cars arrive. Now 2 cars are in the parking lot\nWrong Answer: 2\n\nQuestion: Leah had 32 chocolates and her si

0.8

In [ ]:
data_test.samples[40].question

"Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"

In [ ]:
def infer(prompt_name: str, question: str):
    model = OpenAIModel()
    data = ReasonData.load_from_name("gsm8k", split="train", sample=True)
    sample = ReasonSample(question=question)
    prompt = select_prompter(prompt_name).run(data, sample)
    output = model.run(prompt)

    info = dict(question=question, prompt=prompt, output=output)
    print(json.dumps(info, indent=2))
    # return info

question = """Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"""
infer(prompt_name='contrast_cot',question=question)

{
  "question": "Janet\u2019s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
  "prompt": "Question: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\nExplanation: There are 15 trees originally. Then there were 21 trees after the Grove workers planted some more. So there must have been 21 - 15 = 6 trees that were planted.\nAnswer: 6\nWrong explanation: There are 21 - 15 = 6 trees originally. Then there were 15 trees after the Grove workers planted some more. So there must have been 21 trees that were planted.\nWrong Answer: 21\n\nQuestion: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking l

In [ ]:
!pip install instructor

  Attempting uninstall: jiter
    Found existing installation: jiter 0.7.1
    Uninstalling jiter-0.7.1:
      Successfully uninstalled jiter-0.7.1


In [ ]:
import instructor
import openai
from pydantic import BaseModel, Field
from textwrap import dedent

client = openai.OpenAI(api_key='sk-RsaFA9zGH4bOQl8ExoDoVfVTckmdGuOj2mgCGgRl0WfWQU8u',
                                              base_url="https://api.chatanywhere.tech/v1")
client = instructor.from_openai(client)


class ChainOfThought(BaseModel):
    chain_of_thought: str = Field(description="Incorrect reasoning for the answer")
    correct_answer: str


def contrastive_chain_of_thought(
    query: str,
    context: str,
    example_prompt: str,
    correct_examples: list[str],
    incorrect_examples: list[str],
):
    correct_example_prompt = "\n".join(
        [f"<Explanation>{example}</Explanation>" for example in correct_examples]
    )
    incorrect_example_prompt = "\n".join(
        [
            f"<WrongExplanation>{example}</WrongExplanation>"
            for example in incorrect_examples
        ]
    )
    ""
    return client.chat.completions.create(
        model="gpt-4o-mini",
        response_model=ChainOfThought,
        messages=[
            {
                "role": "system",
                "content": dedent(
                    f"""
            <prompt>
                <role>system</role>
                <context>
                You are an expert question answering AI System.

                You are about to be given some examples of incorrect
                and correct reasoning for a question. You will then
                be asked to correctly reason through another question
                to generate a valid response.
                </context>

                <question>{example_prompt}</question>

                <Explanations>
                    {correct_example_prompt}
                    {incorrect_example_prompt}
                </Explanations>
                <context>{context}</context>
                <question>{query}</question>

            </prompt>
            """
                ),
            }
        ],
    )


if __name__ == "__main__":
    context = """
    James writes a 3-page letter to 2
    different friends twice a week.
    """
    query = "How many pages does James write in a year?"

    sample_question = """
    James has 30 teeth. His dentist drills 4
    of them and caps 7 more teeth than he drills.

    What percentage of James' teeth does the dentist fix?
    """

    incorrect_examples = [
        """James has 30 teeth. The dentist drills and caps some
        teeth. Since drills are normally used on cars and not
        teeth, it's safe to say none of the teeth were actually
        fixed.""",
        """The dentist drills 4 teeth and caps 11 of them, which
        means that he fixes 15 teeth. So we take 15 and multiply
        it by the number of petals on a daisy, and the result is
        30%, which is the percentage of teeth he fixes.""",
    ]

    correct_examples = [
        """The dentist drills 4 teeth, so there are 30 - 4 = 26
        teeth left. The dentist caps 7 more teeth than he drills,
        so he caps 4 + 7 = 11 teeth. Therefore, the dentist fixes
        a total of 4 + 11 = 15 teeth. To find the percentage of
        teeth the dentist fixes, we divide the number of teeth
        fixed by the total number of teeth and multiply by 100:
        15/30 x 100 = 50%"""
    ]

    incorrect_example = [shuffle_equations(example) for example in correct_examples]

    # In kết quả
    print("Incorrect Examples:")
    print(incorrect_example[0])


Incorrect Examples:
The dentist drills 4 teeth, so there are 30 - 4 = 26
        teeth left. The dentist caps 7 more teeth than he drills,
        so he caps 4 + 7 = 11 teeth. Therefore, the dentist fixes
        a total of 4 + 11 = 15 teeth. To find the percentage of
        teeth the dentist fixes, we divide the number of teeth
        fixed by the total number of teeth and multiply by 100:
        15/30 x 100 = 50%


In [ ]:
from textwrap import dedent
correct_examples = [dedent(
    """The dentist drills 4 teeth, so there are 30 - 4 = 26
    teeth left. The dentist caps 7 more teeth than he drills,
    so he caps 4 + 7 = 11 teeth. Therefore, the dentist fixes
    a total of 4 + 11 = 15 teeth. To find the percentage of
    teeth the dentist fixes, we divide the number of teeth
    fixed by the total number of teeth and multiply by 100:
    15/30 x 100 = 50%""")
]
incorrect_example = [shuffle_equations(example) for example in correct_examples]

# In kết quả
print("Incorrect Examples:")
print(incorrect_example[0])

Incorrect Examples:
The dentist drills 4 teeth, so there are 4 + 7 = 11
    teeth left. The dentist caps 7 more teeth than he drills,
    so he caps 4 + 11 = 15 teeth. Therefore, the dentist fixes
    a total of 30 - 4 = 26 teeth. To find the percentage of
    teeth the dentist fixes, we divide the number of teeth
    fixed by the total number of teeth and multiply by 100:
    15/30 x 100 = 50%


In [ ]:
incorrect_examples

["James has 30 teeth. The dentist drills and caps some\n        teeth. Since drills are normally used on cars and not\n        teeth, it's safe to say none of the teeth were actually\n        fixed.",
 'The dentist drills 4 teeth and caps 11 of them, which\n        means that he fixes 15 teeth. So we take 15 and multiply\n        it by the number of petals on a daisy, and the result is\n        30%, which is the percentage of teeth he fixes.',
 'The dentist drills 4 teeth, so there are 30 - 4 = 26\n        teeth left. The dentist caps 7 more teeth than he drills,\n        so he caps 4 + 7 = 11 teeth. Therefore, the dentist fixes\n        a total of 4 + 11 = 15 teeth. To find the percentage of\n        teeth the dentist fixes, we divide the number of teeth\n        fixed by the total number of teeth and multiply by 100:\n        15/30 x 100 = 50%']

In [ ]:
response = contrastive_chain_of_thought(
      query=query,
      context=context,
      example_prompt=sample_question,
      correct_examples=correct_examples,
      incorrect_examples=incorrect_examples,
  )

print(response.model_dump_json(indent=2))
"""
{
  "chain_of_thought": "First, let's determine how many pages James writes per week.
  He writes a 3-page letter to 2 different friends, so for one writing session, he
  writes 3 pages x 2 friends = 6 pages. He does this twice a week, so the total number
    of pages written per week is 6 pages/session x 2 sessions/week = 12 pages/week. \n\n
    Next, we need to find out how many weeks are in a year. There are 52 weeks in a year,
    so we multiply the number of pages James writes per week by the number of weeks in a year:
    12 pages/week x 52 weeks/year = 624 pages/year.\n\nTherefore, James writes 624 pages in a year.",
  "correct_answer": "624"
}
"""

{
  "chain_of_thought": "James writes a 3-page letter twice a week. That means he writes 3 pages x 2 letters = 6 pages each week. A year has 52 weeks, so he writes 6 pages/week x 52 weeks = 312 pages in a year.",
  "correct_answer": "James writes 312 pages in a year."
}


'\n{\n  "chain_of_thought": "First, let\'s determine how many pages James writes per week.\n  He writes a 3-page letter to 2 different friends, so for one writing session, he\n  writes 3 pages x 2 friends = 6 pages. He does this twice a week, so the total number\n    of pages written per week is 6 pages/session x 2 sessions/week = 12 pages/week. \n\n\n    Next, we need to find out how many weeks are in a year. There are 52 weeks in a year,\n    so we multiply the number of pages James writes per week by the number of weeks in a year:\n    12 pages/week x 52 weeks/year = 624 pages/year.\n\nTherefore, James writes 624 pages in a year.",\n  "correct_answer": "624"\n}\n'

Sử dụng để xóa trộn ngữ nghĩa https://spacy.io/models/en#en_core_web_trf

In [ ]:
!python -m spacy download en_core_web_trf # Download the model if it's not already downloaded
!pip install -U pip setuptools wheel
!pip install -U spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 35.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: wheel
    Found existing installation: wheel 0.45.0
    Uninstalling wheel-0.45.0:
      Successfully uninstalled wheel-0.45.0
  Attempting uninstall: setuptools
  

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.1/29.1 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 151.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 140.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: blis
    Found existing installation: blis 0.7.11
    Uninstalling blis-0.7.11:
      Successfully uninstalled blis-0.7.11
  Attempting uninstall: thinc
    Found existing installation: thinc 8.2.5
    Uninstalling thinc-8.2.5:
      Successfully uninstalled thinc-8.2.5
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.5
    Uninstalling spacy-3.7.5:
      Successfully uninstalled spacy-3.7.5
ERROR: pip's dependency resolver does not currently take into account all the packages that

In [ ]:
import spacy


/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_trf' (3.7.3) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpic

In [ ]:
import spacy
from spacy.matcher import Matcher
import random

# Nạp mô hình Spacy
nlp = spacy.load("en_core_web_sm")

# Khởi tạo Matcher
matcher = Matcher(nlp.vocab)

# Mẫu để nhận dạng phương trình
pattern = [
    {"LIKE_NUM": True},  # Số đầu tiên
    {"TEXT": {"IN": ["+", "-", "*", "/"]}},  # Dấu phép toán
    {"LIKE_NUM": True},  # Số thứ hai
    {"TEXT": {"IN": ["="]}},  # Dấu "="
    {"LIKE_NUM": True},  # Kết quả
]
matcher.add("EQUATION", [pattern])

# Văn bản chứa phương trình
correct_examples = [
    """The dentist drills 4 teeth, so there are 30 - 4 = 26
    teeth left. The dentist caps 7 more teeth than he drills,
    so he caps 4 + 7 = 11 teeth. Therefore, the dentist fixes
    a total of 4 + 11 = 15 teeth. To find the percentage of
    teeth the dentist fixes, we divide the number of teeth
    fixed by the total number of teeth and multiply by 100:
    15/30 x 100 = 50%"""
]

# Hàm tạo văn bản trộn phương trình
def shuffle_equations(text):
    doc = nlp(text)
    matches = matcher(doc)

    # Trích xuất các phương trình
    equations = []
    for match_id, start, end in matches:
        span = doc[start:end]
        equations.append(span.text)

    # Xóa trùng lặp và trộn thứ tự phương trình
    unique_equations = list(set(equations))
    shuffled_equations = unique_equations[:]
    random.shuffle(shuffled_equations)

    # Thay thế các phương trình theo thứ tự mới
    incorrect_text = text
    for original, shuffled in zip(unique_equations, shuffled_equations):
        incorrect_text = incorrect_text.replace(original, shuffled, 1)

    # for shuffled in shuffled_equations:
    #     incorrect_text = incorrect_text.replace(f"{{{shuffled}}}", shuffled, 1)

    return incorrect_text

# Tạo incorrect_examples
incorrect_examples = [shuffle_equations(example) for example in correct_examples]

# In kết quả
print("Incorrect Examples:")
print(incorrect_examples[0])


Incorrect Examples:
The dentist drills 4 teeth, so there are 4 + 11 = 15
    teeth left. The dentist caps 7 more teeth than he drills,
    so he caps 4 + 7 = 11 teeth. Therefore, the dentist fixes
    a total of 30 - 4 = 26 teeth. To find the percentage of
    teeth the dentist fixes, we divide the number of teeth
    fixed by the total number of teeth and multiply by 100:
    15/30 x 100 = 50%


In [ ]:
incorrect_examples.append(incorrect_example[0])

In [ ]:
response = contrastive_chain_of_thought(
      query=query,
      context=context,
      example_prompt=sample_question,
      correct_examples=correct_examples,
      incorrect_examples=incorrect_examples,
  )

print(response.model_dump_json(indent=2))
"""
{
  "chain_of_thought": "First, let's determine how many pages James writes per week.
  He writes a 3-page letter to 2 different friends, so for one writing session, he
  writes 3 pages x 2 friends = 6 pages. He does this twice a week, so the total number
    of pages written per week is 6 pages/session x 2 sessions/week = 12 pages/week. \n\n
    Next, we need to find out how many weeks are in a year. There are 52 weeks in a year,
    so we multiply the number of pages James writes per week by the number of weeks in a year:
    12 pages/week x 52 weeks/year = 624 pages/year.\n\nTherefore, James writes 624 pages in a year.",
  "correct_answer": "624"
}
"""

{
  "chain_of_thought": "James writes a 3-page letter to 2 different friends twice a week. This means he writes a total of 3 pages x 2 friends = 6 pages every time he writes. Since he does this twice a week, he writes 6 pages x 2 = 12 pages weekly. There are 52 weeks in a year, so the total pages he writes in a year is 12 pages/week x 52 weeks = 624 pages. Therefore, the total number of pages James writes in a year is 624 pages.",
  "correct_answer": "624 pages"
}


'\n{\n  "chain_of_thought": "First, let\'s determine how many pages James writes per week.\n  He writes a 3-page letter to 2 different friends, so for one writing session, he\n  writes 3 pages x 2 friends = 6 pages. He does this twice a week, so the total number\n    of pages written per week is 6 pages/session x 2 sessions/week = 12 pages/week. \n\n\n    Next, we need to find out how many weeks are in a year. There are 52 weeks in a year,\n    so we multiply the number of pages James writes per week by the number of weeks in a year:\n    12 pages/week x 52 weeks/year = 624 pages/year.\n\nTherefore, James writes 624 pages in a year.",\n  "correct_answer": "624"\n}\n'

In [ ]:
!pip install datasets

  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import json
import random
import instructor
import re
from pathlib import Path
from typing import List, TextIO

import openai
from datasets import load_dataset
# from fire import Fire
from pydantic import BaseModel
from tqdm import tqdm

from textwrap import dedent
import spacy
from spacy.matcher import Matcher
import random

import spacy
from spacy.matcher import Matcher
import random

nlp = spacy.load("en_core_web_trf")
matcher = Matcher(nlp.vocab)

pattern = [
    {"LIKE_NUM": True},  # Số đầu tiên
    {"TEXT": {"IN": ["+", "-", "*", "/"]}},  # Dấu phép toán
    {"LIKE_NUM": True},  # Số thứ hai
    {"TEXT": {"IN": ["="]}},  # Dấu "="
    {"LIKE_NUM": True},  # Kết quả
]
matcher.add("EQUATION", [pattern])


def shuffle_equations(text):
    doc = nlp(text)
    matches = matcher(doc)

    equations = []
    for match_id, start, end in matches:
        span = doc[start:end]
        equations.append(span.text)

    unique_equations = list(set(equations))
    shuffled_equations = unique_equations[:]
    random.shuffle(shuffled_equations)

    # Thay thế các phương trình theo thứ tự mới
    # qua 2 lần
    incorrect_text = text
    for original, shuffled in zip(unique_equations, shuffled_equations):
        incorrect_text = incorrect_text.replace(original, f"{{{shuffled}}}", 1)

    for shuffled in shuffled_equations:
        incorrect_text = incorrect_text.replace(f"{{{shuffled}}}", shuffled, 1)

    return incorrect_text

def remove_substrings_with_double_angle_brackets(input_string):
    # Define the pattern to match substrings within double angled brackets
    pattern = r"<<[^>]+>>"
    # Use the sub() function from the re module to replace matching substrings with an empty string
    result = re.sub(pattern, "", input_string)
    return result


class ReasonSample(BaseModel):
    question: str
    explanation: str = ""
    answer: str = ""
    wrong_explanation: str = ""
    wrong_answer: str = ""
    pred: str = ""


class ReasonData(BaseModel):
    samples: List[ReasonSample]

    @classmethod
    def load(cls, path: str):
        samples = []
        with open(path) as f:
            for line in f:
                raw = json.loads(line)
                samples.append(ReasonSample(**raw))

        return cls(samples=samples)

    @classmethod
    def load_gsm8k_test(cls, path: str = "gsm8k", subset: str = "main", split="test"):
        samples = []
        for raw in load_dataset(path, subset, split=split):
            explanation, answer = raw["answer"].split("####")
            explanation = remove_substrings_with_double_angle_brackets(explanation)
            samples.append(
                ReasonSample(
                    question=raw["question"].strip(),
                    explanation=explanation.strip(),
                    answer=answer.strip(),
                )
            )

        return cls(samples=samples)

    @classmethod
    def load_gsm8k_incoherent_objects(cls, split: str = "test", sample: bool = False):
        q3 = "Originally, Leah had 32 chocolates and her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39 pieces left in total."
        if split == "train" and sample:
            samples = [
                ReasonSample(
                    question="There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?",
                    explanation="There are 15 trees originally. Then there were 21 trees after the Grove workers planted some more. So there must have been 21 - 15 = 6 trees that were planted.",
                    answer="6",
                    wrong_explanation="There are 21 - 15 = 6 trees originally. Then there were 15 trees after the Grove workers planted some more. So there must have been 21 trees that were planted.",
                    wrong_answer="21",
                ),
                ReasonSample(
                    question="If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot??",
                    explanation="There are originally 3 cars. Then 2 more cars arrive. Now 3 + 2 = 5 cars are in the parking lot.",
                    answer="5",
                    wrong_explanation="There are originally 3 + 2 = 5 cars. Then 3 more cars arrive. Now 2 cars are in the parking lot",
                    wrong_answer="2",
                ),
                ReasonSample(
                    question="Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?",
                    explanation="Originally, Leah had 32 chocolates and her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39 pieces left in total.",
                    answer="39",
                    wrong_explanation=shuffle_equations(q3),
                    wrong_answer="42",
                ),
                ReasonSample(
                    question="Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?",
                    explanation="Jason had 20 lollipops originally. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8 lollipops.",
                    answer="8",
                    wrong_explanation="Jason had 20 - 12 = 8 lollipops originally. Then he had 20 after giving some to Denny. So he gave Denny 12 lollipops",
                    wrong_answer="12",
                ),
            ]
            return cls(samples=samples)
        else:
            return cls.load_gsm8k_test()

    @classmethod
    def load_from_name(cls, name: str, **kwargs):
        if name == "gsm8k":
            return cls.load_gsm8k_incoherent_objects(**kwargs)
        else:
            raise KeyError(name)


class Prompter(BaseModel):
    def run(self, data_train: ReasonData, sample_test: ReasonSample) -> str:
        prompt = ""
        for sample in data_train.samples:
            prompt += f"Question: {sample.question}\n"
            prompt += f"Answer: {sample.answer}\n\n"

        prompt += f"Question: {sample_test.question}\n"
        prompt += "Answer: "
        return prompt

    @staticmethod
    def get_answer(text: str) -> str:
        parts = text.split("Answer: ")
        if len(parts) >= 2:
            return parts[1]
        else:
            return text


class ChainThoughtPrompter(Prompter):
    def run(self, data_train: ReasonData, sample_test: ReasonSample) -> str:
        prompt = ""
        for sample in data_train.samples:
            prompt += f"Question: {sample.question}\n"
            prompt += f"Explanation: {sample.explanation}\n"
            prompt += f"Answer: {sample.answer}\n\n"

        prompt += f"Question: {sample_test.question}\n"
        prompt += "Explanation: "
        return prompt

    def get_explanation(self, text: str) -> str:
        assert self is not None
        return text.split("\nAnswer: ")[0]


class ContrastiveChainThoughtPrompter(Prompter):
    def run(self, data_train: ReasonData, sample_test: ReasonSample) -> str:
        prompt = ""
        for sample in data_train.samples:
            prompt += f"Question: {sample.question}\n"
            prompt += f"Explanation: {sample.explanation}\n"
            prompt += f"Answer: {sample.answer}\n"
            prompt += f"Wrong explanation: {shuffle_equations(sample.explanation)}\n"
            prompt += f"Wrong Answer: {sample.wrong_answer}\n\n"

        prompt += f"Question: {sample_test.question}\n"
        prompt += "Explanation: "
        return prompt


def select_prompter(name: str):
    if name == "standard":
        return Prompter()
    if name == "cot":
        return ChainThoughtPrompter()
    if name == "contrast_cot":
        return ContrastiveChainThoughtPrompter()
    else:
        raise KeyError(name)


class OpenAIModel(BaseModel):
    engine : str = ''
    use_azure: bool = False
    timeout: int = 60
    temperature: float = 0.0


    def run(self, prompt: str) -> str:
        output = ""
        error_message = "The response was filtered"
        while not output:
            try:
                key = "engine" if self.use_azure else "model"
                client = openai.OpenAI(api_key='sk-RsaFA9zGH4bOQl8ExoDoVfVTckmdGuOj2mgCGgRl0WfWQU8u',
                                      base_url="https://api.chatanywhere.tech/v1")
                response = client.chat.completions.create(model='gpt-4o-mini',

                    messages=[{"role": "user", "content": prompt}],
                    temperature=self.temperature,  # this is the degree of randomness of the model's output
                )
                if response.choices[0].finish_reason == "content_filter":
                    raise ValueError(error_message)
                output = response.choices[0].message.content
            except Exception as e:
                print(e)
                if error_message in str(e):
                    output = error_message

            if not output:
                print("OpenAIModel request failed, retrying.")

        return output


class Scorer(BaseModel):
    @staticmethod
    def run(pred: str, gold: str) -> float:
        pred = pred.replace("$", "")
        gold = gold.replace("$", "")
        return float(pred.startswith(gold))


def evaluate(
    model: OpenAIModel,
    data_train: ReasonData,
    data_test: ReasonData,
    prompter: Prompter,
    file: TextIO,
) -> dict:
    is_correct = []
    score = 0

    progress = tqdm(data_test.samples)
    sample: ReasonSample

    for sample in progress:
        prompt = prompter.run(data_train, sample)
        raw = model.run(prompt).strip()
        sample.pred = prompter.get_answer(raw)
        is_correct.append(Scorer.run(sample.pred, sample.answer))
        score = sum(is_correct) / len(is_correct)
        progress.set_postfix(score=score)
        print(dict(prompt=prompt, raw=raw, gold=sample.answer, pred=sample.pred))
        print(sample.json(), file=file)

    return dict(score=score)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_trf' (3.7.3) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpic

In [ ]:

def main(
    data_name: 'gsm8k',
    prompt_name: 'contrast_cot',
    data_path: str = None,
    num_shots: int = 4,
    max_test_samples: int = 20,
    seed: int = 0,
    **kwargs,
):
    model = OpenAIModel()
    print(locals())

    data_train = ReasonData.load_from_name(data_name, split="train", sample=True)
    name = f"{prompt_name}_{num_shots=}_{max_test_samples=}"
    data_test = ReasonData.load_from_name(data_name, split="test")

    path_out = f"outputs/{data_name}/{kwargs.get('model_name')}/{name}.json"
    if num_shots >= 0:
        data_train.samples = data_train.samples[:num_shots]
    if 0 < max_test_samples < len(data_test.samples):
        random.seed(seed)
        data_test.samples = random.sample(data_test.samples, k=max_test_samples)
    print(dict(train=len(data_train.samples), test=len(data_test.samples)))

    Path(path_out).parent.mkdir(exist_ok=True, parents=True)
    with open(path_out, "w") as f:
        prompter = select_prompter(prompt_name)
        result = evaluate(model, data_train, data_test, prompter, file=f)

    print(path_out)
    print(result)
    return result["score"]


def infer(prompt_name: str, question: str, **kwargs):
    model = OpenAIModel(**kwargs)
    data = ReasonData.load_from_name("gsm8k", split="train", sample=True)
    sample = ReasonSample(question=question)
    prompt = select_prompter(prompt_name).run(data, sample)
    output = model.run(prompt)

    info = dict(question=question, prompt=prompt, output=output)
    print(json.dumps(info, indent=2))
    # return info



In [ ]:
main('gsm8k','contrast_cot')

{'data_name': 'gsm8k', 'prompt_name': 'contrast_cot', 'data_path': None, 'num_shots': 4, 'max_test_samples': 20, 'seed': 0, 'kwargs': {}, 'model': OpenAIModel(engine='', use_azure=False, timeout=60, temperature=0.0)}
{'train': 4, 'test': 20}


  5%|▌         | 1/20 [00:02<00:56,  3.00s/it, score=1]

{'prompt': 'Question: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\nExplanation: There are 15 trees originally. Then there were 21 trees after the Grove workers planted some more. So there must have been 21 - 15 = 6 trees that were planted.\nAnswer: 6\nWrong explanation: There are 15 trees originally. Then there were 21 trees after the Grove workers planted some more. So there must have been 21 - 15 = 6 trees that were planted.\nWrong Answer: 21\n\nQuestion: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot??\nExplanation: There are originally 3 cars. Then 2 more cars arrive. Now 3 + 2 = 5 cars are in the parking lot.\nAnswer: 5\nWrong explanation: There are originally 3 cars. Then 2 more cars arrive. Now 3 + 2 = 5 cars are in the parking lot.\nWrong Answer: 2\n\nQuestion: Leah had 32 chocolates and her s

 10%|█         | 2/20 [00:05<00:51,  2.84s/it, score=1]

{'prompt': 'Question: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\nExplanation: There are 15 trees originally. Then there were 21 trees after the Grove workers planted some more. So there must have been 21 - 15 = 6 trees that were planted.\nAnswer: 6\nWrong explanation: There are 15 trees originally. Then there were 21 trees after the Grove workers planted some more. So there must have been 21 - 15 = 6 trees that were planted.\nWrong Answer: 21\n\nQuestion: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot??\nExplanation: There are originally 3 cars. Then 2 more cars arrive. Now 3 + 2 = 5 cars are in the parking lot.\nAnswer: 5\nWrong explanation: There are originally 3 cars. Then 2 more cars arrive. Now 3 + 2 = 5 cars are in the parking lot.\nWrong Answer: 2\n\nQuestion: Leah had 32 chocolates and her s

...

 95%|█████████▌| 19/20 [00:57<00:02,  2.94s/it, score=0.842]

{'prompt': 'Question: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\nExplanation: There are 15 trees originally. Then there were 21 trees after the Grove workers planted some more. So there must have been 21 - 15 = 6 trees that were planted.\nAnswer: 6\nWrong explanation: There are 15 trees originally. Then there were 21 trees after the Grove workers planted some more. So there must have been 21 - 15 = 6 trees that were planted.\nWrong Answer: 21\n\nQuestion: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot??\nExplanation: There are originally 3 cars. Then 2 more cars arrive. Now 3 + 2 = 5 cars are in the parking lot.\nAnswer: 5\nWrong explanation: There are originally 3 cars. Then 2 more cars arrive. Now 3 + 2 = 5 cars are in the parking lot.\nWrong Answer: 2\n\nQuestion: Leah had 32 chocolates and her s

100%|██████████| 20/20 [01:00<00:00,  3.03s/it, score=0.85]

{'prompt': 'Question: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\nExplanation: There are 15 trees originally. Then there were 21 trees after the Grove workers planted some more. So there must have been 21 - 15 = 6 trees that were planted.\nAnswer: 6\nWrong explanation: There are 15 trees originally. Then there were 21 trees after the Grove workers planted some more. So there must have been 21 - 15 = 6 trees that were planted.\nWrong Answer: 21\n\nQuestion: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot??\nExplanation: There are originally 3 cars. Then 2 more cars arrive. Now 3 + 2 = 5 cars are in the parking lot.\nAnswer: 5\nWrong explanation: There are originally 3 cars. Then 2 more cars arrive. Now 3 + 2 = 5 cars are in the parking lot.\nWrong Answer: 2\n\nQuestion: Leah had 32 chocolates and her s

0.85

In [ ]:
def infer(prompt_name: str, question: str):
    model = OpenAIModel()
    data = ReasonData.load_from_name("gsm8k", split="train", sample=True)
    sample = ReasonSample(question=question)
    prompt = select_prompter(prompt_name).run(data, sample)
    output = model.run(prompt)

    info = dict(question=question, prompt=prompt, output=output)
    print(json.dumps(info, indent=2))
    # return info

question = """Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"""
infer(prompt_name='contrast_cot',question=question)

{
  "question": "Janet\u2019s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
  "prompt": "Question: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\nExplanation: There are 15 trees originally. Then there were 21 trees after the Grove workers planted some more. So there must have been 21 - 15 = 6 trees that were planted.\nAnswer: 6\nWrong explanation: There are 15 trees originally. Then there were 21 trees after the Grove workers planted some more. So there must have been 21 - 15 = 6 trees that were planted.\nWrong Answer: 21\n\nQuestion: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking l

In [ ]:
question = 'James writes a 3-page letter to 2 different friends twice a week. How many pages does he write a year?'
infer(prompt_name='cot',question=question)

{
  "question": "James writes a 3-page letter to 2 different friends twice a week. How many pages does he write a year?",
  "prompt": "Question: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\nExplanation: There are 15 trees originally. Then there were 21 trees after the Grove workers planted some more. So there must have been 21 - 15 = 6 trees that were planted.\nAnswer: 6\n\nQuestion: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot??\nExplanation: There are originally 3 cars. Then 2 more cars arrive. Now 3 + 2 = 5 cars are in the parking lot.\nAnswer: 5\n\nQuestion: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?\nExplanation: Originally, Leah had 32 chocolates and her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74

In [ ]:
question = 'James writes a 3-page letter to 2 different friends twice a week. How many pages does he write a year?'
infer(prompt_name='contrast_cot',question=question)

{
  "question": "James writes a 3-page letter to 2 different friends twice a week. How many pages does he write a year?",
  "prompt": "Question: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\nExplanation: There are 15 trees originally. Then there were 21 trees after the Grove workers planted some more. So there must have been 21 - 15 = 6 trees that were planted.\nAnswer: 6\nWrong explanation: There are 15 trees originally. Then there were 21 trees after the Grove workers planted some more. So there must have been 21 - 15 = 6 trees that were planted.\nWrong Answer: 21\n\nQuestion: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot??\nExplanation: There are originally 3 cars. Then 2 more cars arrive. Now 3 + 2 = 5 cars are in the parking lot.\nAnswer: 5\nWrong explanation: There are originally 3 cars. Then 2 